In [ ]:
# Assess the similarity between generated DRRs and dataLoaded DRRs from PolyU
import torch
import numpy as np
import scipy as scp
import matplotlib.pyplot as plt


In [ ]:
# Create a dataloader for CXR images
